In [15]:
import json
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from datetime import datetime,timedelta
import random
from itertools import combinations
import matplotlib.pyplot as plt
%matplotlib inline
from langdetect import detect_langs

In [2]:
user_index = json.load(open("../data/user_index.json",'r'))
commitment_index = json.load(open("../data/commitment_index.json",'r'))

In [3]:
max_user_id = max([int(i) for i in user_index.keys()])

In [4]:
for k,v in tqdm(user_index.items()):
    user_index[k]['date'] = datetime.strptime(v['date'], '%d %b %Y')

  0%|          | 0/521632 [00:00<?, ?it/s]

In [5]:
id_for_1_jan_2023 = 722621
for k,v in tqdm(user_index.items()):
    if int(k)<id_for_1_jan_2023 and v['date'].year == 2023:
        user_index[k]['date'] = None

  0%|          | 0/521632 [00:00<?, ?it/s]

In [6]:
last_date = user_index['1']['date']
for i in range(1, max_user_id+1):
    if str(i) in user_index.keys():
        if user_index[str(i)]['date'] == None:
            user_index[str(i)]['date'] = last_date
        else:
            last_date = user_index[str(i)]['date']

In [7]:
for k,v in tqdm(commitment_index.items()):
    commitment_index[k]['start_date'] = datetime.strptime(v['start_date'], '%B %d, %Y')
    commitment_index[k]['end_date'] = datetime.strptime(v['end_date'], '%B %d, %Y')

  0%|          | 0/319928 [00:00<?, ?it/s]

In [8]:
for k,v in tqdm(commitment_index.items()):
    if v['start_date'].year < 2007:
        deta = v['end_date'] - v['start_date']
        commitment_index[k]['start_date'] = user_index[str(v['user_id'])]['date']
        commitment_index[k]['end_date'] = commitment_index[k]['start_date'] + deta

  0%|          | 0/319928 [00:00<?, ?it/s]

Extract data

In [71]:
len(commitment_index)

319928

In [125]:
term_dict = {
    "weight":["weight","diet","fat", "pound", "kg", "calories"],
    "eat":["eat","chocolate", "water", "food", "sugar", "softdrinks", "candy", "desserts", "veggies","gluten", "lactose", "snacking", "coffee","beverage"],
    "exercise":["exercise","run", "walk","race", "cycling","work-out","workout", "bicyclng"],
    "smoking":["smoking"],
    "alcohol":['alcohol','drink'],
    "sleep":["sleep","bed","wake"],
    "read":["read","book"],
    "study":["study","exam","diploma", "phd", "assignment", "math","gmat",'homework'],
    "write":["write","draft", "screenplay","scripts"],
    "mastrubate":["mastrubate","mastrubation","porn","masturbation",'nofap','fap'],
    "meditate":["meditate","journal"],
    "money":["money","finance","saving", "expense", "spending"],
    "nail":["nail"],
    "client":["client","job","business",'network'],
    "procrastinating":["procrastinating","procrastinate","procrastination"],
    "clean":["clean"],
    "internet":["internet", "social media", "electronics",'tv']
}

In [126]:
count_dict = {k:[] for k in term_dict.keys()}

In [127]:
for k,v in tqdm(commitment_index.items()):
    for t,rt in term_dict.items():
        for ct in rt:
            if ct in v['title'].lower():
                count_dict[t].append(k)

  0%|          | 0/319928 [00:00<?, ?it/s]

In [128]:
for k,v in count_dict.items():
    print(f"{k} {len(v)}")

weight 89033
eat 14262
exercise 65993
smoking 8426
alcohol 4950
sleep 6088
read 9020
study 8885
write 4713
mastrubate 2265
meditate 4178
money 2590
nail 495
client 2924
procrastinating 384
clean 1571
internet 1265


In [129]:
sum([len(v) for v in count_dict.values()])

227042

In [130]:
sum([len(v) for v in count_dict.values()])/len(commitment_index)

0.7096659248330874

In [131]:
for k,v in count_dict.items():
    print("-"*100)
    print(k)
    for j in random.sample(v,min(5,len(v))):
        print(commitment_index[j]['title'])

----------------------------------------------------------------------------------------------------
weight
Lose Weight
Lose Weight
Lose Weight
Lose Weight
Lose Weight
----------------------------------------------------------------------------------------------------
eat
Work on product creation for 1 hour...
Spend $0 on food/drink on myself fo...
No crisps, nuts, candy with sugar, ...
No more fancy coffee drinks.
Expense Limiting: Treat Foods
----------------------------------------------------------------------------------------------------
exercise
Exercise Regularly
track and log workout details
Exercise Regularly
Exercise Regularly
Exercise Regularly
----------------------------------------------------------------------------------------------------
smoking
Quit Smoking
Quit Smoking
Quit Smoking
quit smoking
Quit Smoking
----------------------------------------------------------------------------------------------------
alcohol
Drink No Alcohol for 40 days
No Soft Drinks
No Drink

In [132]:
res = list(combinations(count_dict.keys(), 2))
for i in res:
    intersection_count = set.intersection(*[set(count_dict[j]) for j in i])
    if len(intersection_count)>0:
        print("-"*50)
        print(f"{i} {len(intersection_count)}")
        for s in random.sample(list(intersection_count),min(5,len(intersection_count))):
            print(commitment_index[s]['title'])

--------------------------------------------------
('weight', 'eat') 391
Diet & Healthy Eating
lose weight, get toned, eat healthy...
lose weight and eat in moderation
removing sugar from diet
Eat Healthy...& Lose Weight!
--------------------------------------------------
('weight', 'exercise') 269
Lose weight, exercise
Follow a specific diet and exercise...
Bodyweight workout
Weight and Exercise
Exercise/ Weight Loss/Control Diabe...
--------------------------------------------------
('weight', 'smoking') 7
LOSE WEIGHT AND QUIT SMOKING
Lose 20kgs and quit smoking
quit smoking & lose weight
quit smoking & lose weight
Quit smoking, lose weight, do well ...
--------------------------------------------------
('weight', 'alcohol') 60
Drinking-Gym-Weight Loss-Moods
I will not drink diet soda, eat dee...
No drinking and lose body fat
don't drink calories (soda, juice, ...
Diary my food, drink, and weight fo...
--------------------------------------------------
('weight', 'sleep') 9
lose weig

--------------------------------------------------
('read', 'write') 355
Read and write for 4h
Writing the first draft of my book
Write a 300 page book by 12/31/10
Write Chapter 1 of my New book by C...
Write the book
--------------------------------------------------
('read', 'mastrubate') 2
No Facebook and No pornography
give up netflix, facebook, porn, an...
--------------------------------------------------
('read', 'meditate') 37
Read 1 ABA journal article a week
Read more journal articles
Read and journal every day
Read, Workout, Meditate
Wake up at 5am: Read/Meditate/Sourc...
--------------------------------------------------
('read', 'money') 17
read a book on finance/economy
To read non finance stuff for an ho...
Read two Finance news everday
Have Beer Money Strategy(ies) Ready...
Complete Personal Finance text book...
--------------------------------------------------
('read', 'nail') 1
Running, Weights, No Nails, Reading...
--------------------------------------------------


In [133]:
res = list(combinations(count_dict.keys(), 3))
for i in res:
    intersection_count = set.intersection(*[set(count_dict[j]) for j in i])
    if len(intersection_count)>0:
        print("-"*50)
        print(f"{i} {len(intersection_count)}")
        for s in random.sample(list(intersection_count),min(5,len(intersection_count))):
            print(commitment_index[s]['title'])

--------------------------------------------------
('weight', 'eat', 'exercise') 13
Lose weight, eat healthy, exercise ...
Eat 1700 Calories and walk 5000 ste...
create moderated diet/exercise prog...
Eat better, lose weight, exercise
follow diet (exercise, food, budget...
--------------------------------------------------
('weight', 'eat', 'alcohol') 7
Diary my food, drink, and weight fo...
Drink half my weight in water daily...
Drink my weight in ounces of water ...
Diary my food, drink, and weight fo...
I will not drink diet soda, eat dee...
--------------------------------------------------
('weight', 'eat', 'read') 1
Sugar-free, books and pounding the ...
--------------------------------------------------
('weight', 'eat', 'money') 1
Create a spending diet
--------------------------------------------------
('weight', 'eat', 'clean') 6
Eating clean/Lose weight by Oct 1
Lose weight and eat clean
Lose weight, Eat clean, get definit...
Eat Clean and lost weight.
Clean eating for healt

In [134]:
res = list(combinations(count_dict.keys(), 4))
for i in res:
    intersection_count = set.intersection(*[set(count_dict[j]) for j in i])
    if len(intersection_count)>0:
        print("-"*50)
        print(f"{i} {len(intersection_count)}")
        for s in random.sample(list(intersection_count),min(5,len(intersection_count))):
            print(commitment_index[s]['title'])

--------------------------------------------------
('weight', 'exercise', 'read', 'nail') 1
Running, Weights, No Nails, Reading...
--------------------------------------------------
('eat', 'exercise', 'sleep', 'write') 1
Sleep exercise write eat healthy


# where breakes

In [135]:
classified = set([j for v in count_dict.values() for j in v])

In [136]:
len(classified)

217227

In [137]:
not_classified = set(commitment_index.keys())-classified

In [138]:
len(not_classified)

102701

In [139]:
for k in random.sample(list(not_classified),10):
    print(commitment_index[k]['title'])

Get a hair cut
The Basement
BBG
Bring Grades up!
Daily Art
Зарабатывать $400 в месяц...
Aprender Inglês
ROS invite test 3
I need to maintain at least a 3.0 G...
nine-minute mile
